In [2]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1

In [3]:
import numpy as np
import math
import pandas as pd
from typing import Union, List
import datetime
from pycmqlib3.utility import misc
from pycmqlib3.utility.dbaccess import *
import pycmqlib3.analytics.data_handler as dh
import pathlib
from pycmqlib3.utility.process_wt_data import *
from wtpy.wrapper import WtDataHelper
# from pycmqlib3.utility import dataseries
# from pycmqlib3.analytics.tstool import *
# from pycmqlib3.analytics.btmetrics import *


In [4]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [6]:
from misc_scripts.daily_update_job import update_data_from_xl
update_data_from_xl()

saving data for ifind_data.xlsx:const, total cols:27


C:\dev\pyktrader3\pycmqlib3\utility\dbaccess.py:76: SAWarning: Can't validate argument 'replace_string'; can't locate any SQLAlchemy dialect named 'replace'
  conn.execute(table.table.insert(replace_string=""), data)


saving data for ifind_data.xlsx:daily, total cols:139
saving data for ifind_data.xlsx:weekly, total cols:139
saving data for ifind_data.xlsx:sector, total cols:38


In [9]:
base_folder = "C:/dev/wtdev/storage/his"

zip_wt_dir(base_folder, 'C:/dev/data/wt_data_230626.zip', cutoff=datetime.date(2023,6,20))

In [ ]:
base_folder = "C:/dev/wtdev/storage/his"
update_folder = "C:/dev/data/his"
cutoff = datetime.date(2023,6,20)
update_wt_store(base_folder=base_folder, update_folder = update_folder, cutoff=cutoff)

day-CFFEX
day-DCE
day-CZCE
day-SHFE
day-INE
day-GFEX
min1-CFFEX
min1-DCE


In [ ]:
start_date = datetime.date(2022,12,22)
end_date = datetime.date(2022,12,23)
src_folder = "C:/dev/wtdev/storage/his"
dst_folder = "C:/dev/wtdev/storage/his"
cutoff_date = 20221223

save_bars_to_wt_store(exchange_list=['GFEX'],
                        start_date=start_date,
                        end_date=end_date,
                        cutoff_date=cutoff_date,
                        process_min=True,
                        process_day=True,
                        src_folder=src_folder,
                        dst_folder=dst_folder)

In [ ]:
start_date = datetime.date(2022,12,22)
end_date = datetime.date(2022,12,22)
dst_folder = "C:/dev/wtdev/storage/his"

save_ticks_to_wt_store(
        exchange_list=['GFEX'],
        start_date=start_date,
        end_date=end_date,
        dst_folder=dst_folder,
        exclusion_list=[])

In [7]:
period = 'day'
exch = 'DCE'
cont = 'i2309'
src_folder = 'c:/dev/wtdev/storage/his'
filename = '%s/%s/%s/%s.dsb' % (src_folder, period, exch, cont)
dtHelper = WtDataHelper()
curr_df = dtHelper.read_dsb_bars(filename).to_df()
curr_df.tail(10)

,date,bartime,open,high,low,close,settle,money,volume,hold,diff
199000000000,20230609,199000000000,793.5,812.5,791.5,812.0,800.0,8.100004e+10,1012242.0,926918.0,14605.0
199000000000,20230612,199000000000,810.0,811.0,782.0,785.5,796.0,8.156051e+10,1024567.0,865473.0,-61445.0
199000000000,20230613,199000000000,784.5,803.0,781.0,801.5,792.5,7.826362e+10,987231.0,849235.0,-16238.0
199000000000,20230614,199000000000,804.5,811.0,797.5,804.5,804.0,6.460089e+10,803202.0,856715.0,7480.0
199000000000,20230615,199000000000,809.5,820.5,804.5,815.5,814.0,6.655331e+10,817467.0,852864.0,-3851.0
199000000000,20230616,199000000000,814.5,818.5,802.0,815.0,810.5,6.736205e+10,830930.0,840351.0,-12513.0
199000000000,20230619,199000000000,815.0,825.0,805.0,809.5,814.0,7.341365e+10,901580.0,828825.0,-11526.0
199000000000,20230620,199000000000,810.0,812.5,799.0,806.5,805.5,5.483830e+10,680595.0,825941.0,-2884.0
199000000000,20230621,199000000000,803.5,810.5,791.5,797.5,800.5,7.189717e+10,898145.0,804577.0,-21364.0
199000000000,20230626,199000000000,785.0,799.5,784.0,797.0,791.5,4.143554e+10,523391.0,833933.0,29356.0


In [ ]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

In [14]:
sdate = datetime.date(2000,1,1)
edate = datetime.date(2022,9,30)

cnx = connect(**dbconfig)
cont = 'TA301'
mdf = load_min_data_to_df(cnx, 'fut_min', cont, sdate, edate, index_col = None, shift_datetime = True)

# mdf['rdate'] = mdf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
# mdf['time'] = 0
# mdf['hold'] = mdf['openInterest']
# mdf['diff'] = mdf['hold'].diff().fillna(0)
# mdf['money'] = mdf['close'] * mdf['volume']
# mdf['vol'] = mdf['volume']
col_list = ['date', 'time', 'open', 'high', 'low', 'close', 'money', 'vol', 'hold', 'diff']
# mdf = mdf[col_list]            
mdf = conv_min_data(mdf)
print(mdf[col_list])

           date        time    open    high     low   close     money  vol  \
0      20120120  2201201015  8780.0  8780.0  8780.0  8780.0   17560.0    2   
1      20120130  2201300901  8824.0  8824.0  8824.0  8824.0   17648.0    2   
2      20120131  2201311008  9016.0  9016.0  9016.0  9016.0   18032.0    2   
3      20120131  2201311046  9016.0  9070.0  9016.0  9070.0   18140.0    2   
4      20120131  2201311055  9070.0  9070.0  9066.0  9066.0   18132.0    2   
...         ...         ...     ...     ...     ...     ...       ...  ...   
38845  20130115  2301151001  8874.0  9000.0  8874.0  9000.0   36000.0    4   
38846  20130115  2301151103  9000.0  9022.0  9000.0  9022.0   36088.0    4   
38847  20130116  2301161500  9022.0  9022.0  8850.0  8850.0  141600.0   16   
38848  20130117  2301170929  9078.0  9078.0  9078.0  9078.0  108936.0   12   
38849  20130117  2301171037  9078.0  9078.0  8906.0  8906.0   35624.0    4   

       hold   diff  
0         2    0.0  
1         2    0.0  


In [ ]:
ddf = load_daily_data_to_df(cnx, 'fut_daily', cont, datetime.date(2000,1,1), edate, index_col='date', field = 'instID', date_as_str = False)
ddf = ddf.rename(columns = {'openInterest': 'open_interest'}).reset_index()
ddf['pre_settle'] = ddf['settle'].shift(1)
ddf['pre_interest'] = ddf['open_interest'].shift(1)
ddf['pre_close'] = ddf['close'].shift(1)
ddf['date'] = ddf['date'].astype('datetime64').dt.strftime('%Y%m%d').astype('int64')
ddf = ddf.set_index('date')